# <center> IBM Capstone Final Assigment: The Battle of the Neighborhoods</center>



## Project: Finding which neighborhoods in Toronto have the biggest similarities with New York neighborhoods 

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests

# Using the BeautifulSoup package 
# Web scraping packages
from bs4 import BeautifulSoup

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Pandas, Numpy, Json Requests, Matplotlib, folium, BeautifulSoup, Kmeans imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

In [3]:
!conda install -c conda-forge geocoder --yes
print("Installation Done!")
import geocoder # import geocoder
print("Library GeoCoder imported!")

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geocoder-1.38.1            |             py_1          53 KB  conda-forge
    ratelim-0.1.6              |             py_2           6 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          59 KB

The following NEW packages will be INSTALLED:

    geocoder: 1.38.1-py_1 conda-forge
    ratelim:  0.1.6-py_2  conda-forge


geocoder-1.38.1      | 53 KB     | ##################################### | 100% 
ratelim-0.1.6        | 6 KB      | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Installation Done!
Library GeoCoder imported!


In [4]:
# define geocoder function
def get_geocoder(postal_code_from_df):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code_from_df.strip()))
        lat_lng_coords = g.latlng
        latitude = lat_lng_coords[0]
        longitude = lat_lng_coords[1]
    return latitude,longitude

In [5]:
# FourSquare credentials
CLIENT_ID = 'X0E1VUA0ZKGAMJQ21IWNB2GT5VTQOM2R5HFNZV2GMMQVT2GB' # your Foursquare ID
CLIENT_SECRET = '1NXDFKLVY0ECYL13Y3JKFWWY4ENK004O2KY40PCKRG40X2Q3' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Foursquare API credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Foursquare API credentails:
CLIENT_ID: X0E1VUA0ZKGAMJQ21IWNB2GT5VTQOM2R5HFNZV2GMMQVT2GB
CLIENT_SECRET:1NXDFKLVY0ECYL13Y3JKFWWY4ENK004O2KY40PCKRG40X2Q3


## New York City Data

1. Importing Neighborhood data

In [4]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [5]:
# Importing file
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

neighborhoods_data = newyork_data['features']

column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
nyc_df = pd.DataFrame(columns=column_names)
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    nyc_df = nyc_df.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

print('NYC data shape: ' , nyc_df.shape)
nyc_df.head()

NYC data shape:  (306, 4)


,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [6]:
print('The NYC dataframe has {} boroughs and {} neighborhoods.'.format(
        len(nyc_df['Borough'].unique()),
        nyc_df.shape[0]
    )
)

The NYC dataframe has 5 boroughs and 306 neighborhoods.


In [12]:
nyc_success = nyc_df.loc[nyc_df.Neighborhood.isin(['Bedford Stuyvesant','Flatbush', 'Financial District', 'Boerum Hill', 'North Side','Hamilton Heights', 'Manhattanville', 'Murray Hill', 'Hunters Point',\
                                                             'Long Island City', 'Arden Heights'])].reset_index(drop=True) 
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

map_nyc_success = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, long, borough, neigh in zip(nyc_success['Latitude'], nyc_success['Longitude'], nyc_success['Borough'], nyc_success['Neighborhood']):
    label = '{}, {}'.format(neigh, borough)
    popup = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=popup,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_nyc_success)
    
map_nyc_success

2. Obtain Venue information with FourSquare

In [9]:
# Use FourSquare API to get venue information
radius = 500
LIMIT = 100
def getNearbyVenues(Borough, Nhood, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for bgh, name, lat, lng in zip(Borough, Nhood, latitudes, longitudes):
        #print(Nhood)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            bgh, 
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough', 
                  'Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


In [10]:
nyc_venues = getNearbyVenues(Borough=nyc_df['Borough'], 
                             Nhood=nyc_df['Neighborhood'],
                                   latitudes=nyc_df['Latitude'],
                                   longitudes=nyc_df['Longitude']
                                  )


print('Shape of NYC venues: ' , nyc_venues.shape)
print('The NYC venues dataframe has {} unique Neighborhood.'.format(
        len(nyc_venues['Neighborhood'].unique())
    )
)
print('In NYC there are {} venues uniques categories.'.format(len(nyc_venues['Venue Category'].unique())))


Shape of NYC venues:  (10408, 8)
The NYC venues dataframe has 300 unique Neighborhood.
In NYC there are 427 venues uniques categories.


In [11]:
nyc_venues.head()

,Borough,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bronx,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,Bronx,Wakefield,40.894705,-73.847201,Rite Aid,40.896649,-73.844846,Pharmacy
2,Bronx,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop
3,Bronx,Wakefield,40.894705,-73.847201,Cooler Runnings Jamaican Restaurant Inc,40.898276,-73.850381,Caribbean Restaurant
4,Bronx,Wakefield,40.894705,-73.847201,Shell,40.894187,-73.845862,Gas Station


3. Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category so we can identify the most common

In [13]:
# one hot encoding
nyc_onehot = pd.get_dummies(nyc_venues[['Venue Category']], prefix="", prefix_sep="")

# add borough, neighborhood, lat and long column back to dataframe
nyc_onehot['Borough'] = nyc_venues['Borough'] 
nyc_onehot['Neighborhood'] = nyc_venues['Neighborhood'] 
nyc_onehot['Neighborhood Latitude'] = nyc_venues['Neighborhood Latitude'] 
nyc_onehot['Neighborhood Longitude'] = nyc_venues['Neighborhood Longitude'] 

# move borough, neighborhood, lat and long column to the first column
fixed_columns = list(nyc_onehot.columns[-4:]) + list(nyc_onehot.columns[:-4])
nyc_onehot = nyc_onehot[fixed_columns]

nyc_grouped = nyc_onehot.groupby(['Borough','Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude']).mean().reset_index()
print("Shape of Dataframe for NYC Neiborhoods Grouped by Venues is - ",nyc_grouped.shape)
nyc_grouped.head()


Shape of Dataframe for NYC Neiborhoods Grouped by Venues is -  (304, 430)


,Borough,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Child Care Service,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Basketball Court,College Bookstore,College Cafeteria,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dosa Place,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Fabric Shop,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,Office,Oper

## Toronto City Data

1. Scrape Toronto Zip Code Data from Wikipedia

In [14]:
# Load URL that need in the dataframe
Table_url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
res = requests.get(Table_url)
soup = BeautifulSoup(res.text, 'html.parser')
table = soup.find('table',{'class':'wikitable sortable'}).tbody
table_rows=table.find_all('tr')

pc_list = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        pc_list.append(row)

df_pcs = pd.DataFrame(pc_list, columns=["PostalCode", "Borough", "Neighborhood"])
print("Shape of Dataframe is - ",df_pcs.shape)
df_pcs.head()

Shape of Dataframe is -  (288, 3)


,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


2. Clean the data

In [15]:
#Ignore cells with a borough that is Not assigned.
df_pcs = df_pcs[df_pcs.Borough!="Not assigned"]

#If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
df_pcs['Neighborhood'] = np.where(df_pcs['Neighborhood'] == 'Not assigned', df_pcs['Borough'], df_pcs['Neighborhood'])

# Group neighborhoods with the same postal code area
Toronto_gr = pd.DataFrame(df_pcs.groupby(['PostalCode','Borough'])['Neighborhood'].apply(','.join).reset_index())

print("Shape of Dataframe with Toronto grouped Neiborhoods is - ",Toronto_gr.shape)
Toronto_gr.head(11)

Shape of Dataframe with Toronto grouped Neiborhoods is -  (103, 3)


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


3. Add geographic coordenates

In [16]:
# Add columns
Toronto_gr['Latitude'], Toronto_gr['Longitude'] = zip(*Toronto_gr['PostalCode'].apply(get_geocoder))
Toronto_gr.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.811525,-79.195517
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.785730,-79.158750
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.765690,-79.175256
3,M1G,Scarborough,Woburn,43.768359,-79.217590
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440


In [17]:
print('The Toronto dataframe has {} boroughs and {} PostalCodes.'.format(
        len(Toronto_gr['Borough'].unique()),
        Toronto_gr.shape[0]
    )
)

The Toronto dataframe has 11 boroughs and 103 PostalCodes.


4. Obtain Venue information with FourSquare

In [18]:
Toronto_venues = getNearbyVenues(Borough=Toronto_gr['Borough'], 
                                 Nhood=Toronto_gr['Neighborhood'],
                                   latitudes=Toronto_gr['Latitude'],
                                   longitudes=Toronto_gr['Longitude']
                                  )

#Saving FourSquare result, since seem to run out of cuota
export_T_csv = Toronto_venues.to_csv (r'C:\Users\porellana\Desktop\Toronto_venues.csv', index = None, header=True) #Don't forget to add '.csv' at the end of the path

print('Shape of Toronto venues: ' , Toronto_venues.shape)
print('The Toronto venues dataframe has {} unique Neighborhood.'.format(
        len(Toronto_venues['Neighborhood'].unique())
    )
)
print('In Toronto there are {} venues uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))


Shape of Toronto venues:  (2477, 8)
The Toronto venues dataframe has 102 unique Neighborhood.
In Toronto there are 261 venues uniques categories.


4. Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category so we can identify the most common

In [ ]:
# Reloading Foursquare results
Toronto_venues = pd.read_csv('C:\Users\porellana\Desktop\Toronto_venues.csv')


In [19]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add borough, neighborhood, lat and long column back to dataframe
Toronto_onehot['Borough'] = Toronto_venues['Borough'] 
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 
Toronto_onehot['Neighborhood Latitude'] = Toronto_venues['Neighborhood Latitude'] 
Toronto_onehot['Neighborhood Longitude'] = Toronto_venues['Neighborhood Longitude'] 

# move borough, neighborhood, lat and long column to the first column
fixed_columns = list(Toronto_onehot.columns[-4:]) + list(Toronto_onehot.columns[:-4])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_grouped = Toronto_onehot.groupby(['Borough','Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude']).mean().reset_index()
print("Shape of Dataframe for Toronto Neiborhoods Grouped by Venues is - ",Toronto_grouped.shape)
Toronto_grouped.head()



Shape of Dataframe for Toronto Neiborhoods Grouped by Venues is -  (102, 264)


,Borough,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Yoga Studio,ATM,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Moving Target,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Peruvian Restaurant,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shopping Mall,Skating Rink,Ski Chalet,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Tunnel,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Central Toronto,Davisville,43.703395,-79.385964,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0

## Combine venues with frequency by neighboorhood from both cities

In [20]:
venues_comb = pd.concat([nyc_grouped, Toronto_grouped], ignore_index=True, sort=False)

venues_comb['City'] = venues_comb['Neighborhood Latitude'].apply(lambda x: 'Toronto' if x > 42 else 'NYC')
print('Shape of the venues information grouped by neighborhood of both cities combined is:', venues_comb.shape)
venues_comb.sort_values(by='Neighborhood').head(10)


Shape of the venues information grouped by neighborhood of both cities combined is: (406, 458)


,Borough,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Child Care Service,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Basketball Court,College Bookstore,College Cafeteria,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dosa Place,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Fabric Shop,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,Office,Oper

In [24]:
#Saving 
exp_venues_comb=venues_comb.to_csv(r'C:\Users\porellana\Desktop\venues_comb.csv', index = None, header=True) #Don't forget to add '.csv' at the end of the path

## Clustering

1. Cleaning data for NaN

In [21]:
venues_comb = venues_comb.fillna(0)

2. Let's make 5 kluster in the combines Cities data

In [22]:
kclusters = 5

clustering = venues_comb.drop(['Borough', 'Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude', 'City'], 1)



In [23]:
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0], dtype=int32)

In [24]:
# kmeans.labels_[0:10]
venues_comb_cl = venues_comb.copy()

# add clustering labels
venues_comb_cl["ClusterLabel"] = kmeans.labels_

fixed_columns = list(venues_comb_cl.columns[-2:]) + list(venues_comb_cl.columns[:-2])
venues_comb_cl = venues_comb_cl[fixed_columns]

print('Shape of the Venue Combined Clustered is: ',venues_comb_cl.shape)
venues_comb_cl.head()

Shape of the Venue Combined Clustered is:  (406, 459)


,City,ClusterLabel,Borough,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Child Care Service,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Basketball Court,College Bookstore,College Cafeteria,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dosa Place,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Fabric Shop,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle

3. Sort the venues in descending order and put results in a dataframe for top 5 by Neighborhood

In [25]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[7:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [26]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['City', 'ClusterLabel', 'Borough', 'Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
venues_comb_sorted = pd.DataFrame(columns=columns)
venues_comb_sorted['City'] = venues_comb_cl['City']
venues_comb_sorted['ClusterLabel'] = venues_comb_cl['ClusterLabel']
venues_comb_sorted['Borough'] = venues_comb_cl['Borough']
venues_comb_sorted['Neighborhood'] = venues_comb_cl['Neighborhood']
venues_comb_sorted['Neighborhood Latitude'] = venues_comb_cl['Neighborhood Latitude']
venues_comb_sorted['Neighborhood Longitude'] = venues_comb_cl['Neighborhood Longitude']


for ind in np.arange(venues_comb_cl.shape[0]):
    venues_comb_sorted.iloc[ind, 6:] = return_most_common_venues(venues_comb_cl.iloc[ind, :], num_top_venues)


venues_comb_sorted.head()

,City,ClusterLabel,Borough,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,NYC,0,Bronx,Allerton,40.865788,-73.859319,Pizza Place,Supermarket,Pharmacy,Chinese Restaurant,Bus Station
1,NYC,0,Bronx,Baychester,40.866858,-73.835798,Donut Shop,Discount Store,Fried Chicken Joint,Fast Food Restaurant,Spanish Restaurant
2,NYC,0,Bronx,Bedford Park,40.870185,-73.885512,Diner,Pizza Place,Chinese Restaurant,Mexican Restaurant,Sandwich Place
3,NYC,0,Bronx,Belmont,40.857277,-73.888452,Italian Restaurant,Pizza Place,Deli / Bodega,Bakery,Bank
4,NYC,0,Bronx,Bronxdale,40.852723,-73.861726,Italian Restaurant,Latin American Restaurant,Performing Arts Venue,Gym,Breakfast Spot


In [27]:
print(venues_comb_sorted.shape)

(406, 11)


In [39]:
# Check for clusters NA
nan_rows = venues_comb_sorted[venues_comb_sorted['ClusterLabel'].isnull()]
print(nan_rows.shape)
nan_rows.head()

(0, 11)


,City,ClusterLabel,Borough,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue


4. We now can visualize the clusters

In [28]:
avg_lat = 42.3
avg_long = -76.7
map_clusters = folium.Map(location=[avg_lat, avg_long], zoom_start=8)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, bor, neigh, cluster in zip(venues_comb_cl['Neighborhood Latitude'], venues_comb_cl['Neighborhood Longitude'], venues_comb_cl['Borough'], venues_comb_cl['Neighborhood'], venues_comb_cl['ClusterLabel']):
    label = folium.Popup('{} : {} - Cluster {}'.format(bor, neigh, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [29]:
toronto_clusters = venues_comb_cl[venues_comb_sorted['City'] == 'Toronto'].reset_index(drop=True) 
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_ontario")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, long, borough, neigh, cluster in zip(toronto_clusters['Neighborhood Latitude'], toronto_clusters['Neighborhood Longitude'], toronto_clusters['Borough'], toronto_clusters['Neighborhood'], toronto_clusters['ClusterLabel']):
    label = "{} ({}): {}".format(borough, neigh, cluster)
    popup = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=popup,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
    
map_toronto

In [30]:
nyc_clusters = venues_comb_sorted[venues_comb_cl['City'] == 'NYC'].reset_index(drop=True) 
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

map_nyc = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, long, borough, neigh, cluster in zip(nyc_clusters['Neighborhood Latitude'], nyc_clusters['Neighborhood Longitude'], nyc_clusters['Borough'], nyc_clusters['Neighborhood'], nyc_clusters['ClusterLabel']):
    label = "{} ({}): {}".format(borough, neigh, cluster)
    popup = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=popup,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7,
        parse_html=False).add_to(map_nyc)
    
map_nyc

3. Explore the clusters

In [31]:
venues_comb_sorted.loc[venues_comb_sorted['ClusterLabel'] == 0, venues_comb_sorted.columns[[0]+[1] + [2] +[3]+ list(range(6, venues_comb_sorted.shape[1]))]]

,City,ClusterLabel,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,NYC,0,Bronx,Allerton,Pizza Place,Supermarket,Pharmacy,Chinese Restaurant,Bus Station
1,NYC,0,Bronx,Baychester,Donut Shop,Discount Store,Fried Chicken Joint,Fast Food Restaurant,Spanish Restaurant
2,NYC,0,Bronx,Bedford Park,Diner,Pizza Place,Chinese Restaurant,Mexican Restaurant,Sandwich Place
3,NYC,0,Bronx,Belmont,Italian Restaurant,Pizza Place,Deli / Bodega,Bakery,Bank
4,NYC,0,Bronx,Bronxdale,Italian Restaurant,Latin American Restaurant,Performing Arts Venue,Gym,Breakfast Spot
5,NYC,0,Bronx,Castle Hill,Pizza Place,Diner,Deli / Bodega,Bank,Market
6,NYC,0,Bronx,City Island,Harbor / Marina,Seafood Restaurant,Thrift / Vintage Store,Pharmacy,Music Venue
7,NYC,0,Bronx,Claremont Village,Bus Station,Pizza Place,Grocery Store,Restaurant,Bakery
9,NYC,0,Bronx,Co-op City,Bus Station,Baseball Field,Park,Ice Cream Shop,Pizza Place
10,NYC,0,Bronx,Concourse,Pizza Place,Supermarket,Deli / Bodega,Bus Station,Pharmacy


#### Cluster 1 consist of Neighborhoods with casual eating locations such as Pizza Place and Coffee Shops as the most common venues

In [32]:
# Cluster 1 detail
c1=venues_comb_cl[venues_comb_cl['ClusterLabel'] ==0]
#c3df=venues_comb_sorted.loc[venues_comb_sorted['ClusterLabel'] == 3, venues_comb_sorted.columns[[0]+[1] + [2] +[3]+ list(range(6, venues_comb_sorted.shape[1]))]]
print(c1.shape)
c1.head()


(347, 459)


,City,ClusterLabel,Borough,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Child Care Service,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Basketball Court,College Bookstore,College Cafeteria,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dosa Place,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Fabric Shop,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle

In [33]:
#print(c3df.shape)
c1l=(c1.sum(axis = 0, skipna = True)[6:])*100
c1l_2 = c1l.sort_values(ascending = False)
print(c1l_2 )
#
#

Pizza Place                                 1657.32
Coffee Shop                                 1051.02
Deli / Bodega                               985.913
Pharmacy                                    885.593
Italian Restaurant                          835.143
Chinese Restaurant                          760.467
Sandwich Place                              733.814
Grocery Store                               686.634
Donut Shop                                   675.17
Bakery                                      660.197
Café                                        638.366
Bank                                         622.95
Bar                                         581.263
Park                                        579.772
Supermarket                                 559.992
Fast Food Restaurant                        558.373
Bus Station                                 458.796
Playground                                  454.664
Gym                                         437.602
Caribbean Re

In [34]:
venues_comb_sorted.loc[venues_comb_sorted['ClusterLabel'] == 1, venues_comb_sorted.columns[[0]+[1] + [2] +[3]+ list(range(6, venues_comb_sorted.shape[1]))]]

,City,ClusterLabel,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
8,NYC,1,Bronx,Clason Point,Park,Pool,Business Service,Boat or Ferry,Grocery Store
39,NYC,1,Bronx,Riverdale,Park,Bus Station,Bank,Home Service,Plaza
168,NYC,1,Queens,Bayswater,Park,Playground,Tunnel,Fish & Chips Shop,Fish Market
233,NYC,1,Queens,South Ozone Park,Park,Fast Food Restaurant,Deli / Bodega,Sandwich Place,Food Truck
253,NYC,1,Staten Island,Chelsea,Park,Steakhouse,Sandwich Place,Italian Restaurant,Fountain
305,Toronto,1,Central Toronto,Davisville North,Park,Clothing Store,Gym,Breakfast Spot,Food & Drink Shop
307,Toronto,1,Central Toronto,"Forest Hill North,Forest Hill West",Park,Construction & Landscaping,Field,Fish & Chips Shop,Fish Market
309,Toronto,1,Central Toronto,"Moore Park,Summerhill East",Park,Gym,Tennis Court,Convenience Store,Gastropub
310,Toronto,1,Central Toronto,North Toronto West,Park,Garden,Gym Pool,Playground,Frame Store
327,Toronto,1,Downtown Toronto,Rosedale,Park,Building,Bank,Playground,Fried Chicken Joint


#### Cluster 2 consist of Neighborhoods with Parks and open Fields most common venues

In [35]:
venues_comb_sorted.loc[venues_comb_sorted['ClusterLabel'] == 2, venues_comb_sorted.columns[[0]+[1] + [2] +[3]+ list(range(6, venues_comb_sorted.shape[1]))]]

,City,ClusterLabel,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
311,Toronto,2,Central Toronto,Roselawn,Home Service,Tunnel,Field,Fish & Chips Shop,Fish Market
392,Toronto,2,Scarborough,"Rouge,Malvern",Construction & Landscaping,Home Service,Tunnel,Fish & Chips Shop,Fish Market


#### Cluster 3 consist of Neighborhoods with Construction and Home improvement as the most common venues

In [36]:
venues_comb_sorted.loc[venues_comb_sorted['ClusterLabel'] == 3, venues_comb_sorted.columns[[0]+[1] + [2] +[3]+ list(range(6, venues_comb_sorted.shape[1]))]]

,City,ClusterLabel,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
231,NYC,3,Queens,Somerville,Park,Gift Shop,Filipino Restaurant,Fish & Chips Shop,Fish Market
296,NYC,3,Staten Island,Todt Hill,Park,Gift Shop,Filipino Restaurant,Fish & Chips Shop,Fish Market


#### Cluster 4 consist of Neighborhoods with Gift Shops and Parks as the most common venues - Probably touristic attractions close by

In [37]:
venues_comb_sorted.loc[venues_comb_sorted['ClusterLabel'] == 4, venues_comb_sorted.columns[[0]+[1] + [2] +[3]+ list(range(6, venues_comb_sorted.shape[1]))]]

,City,ClusterLabel,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
102,NYC,4,Brooklyn,Ocean Hill,Deli / Bodega,Bus Stop,Grocery Store,Southern / Soul Food Restaurant,Fried Chicken Joint
177,NYC,4,Queens,Brookville,Deli / Bodega,Market,Tunnel,French Restaurant,Fish & Chips Shop
214,NYC,4,Queens,Neponsit,Beach,Bus Stop,Tunnel,French Restaurant,Fish Market
244,NYC,4,Staten Island,Arden Heights,Coffee Shop,Bus Stop,Pharmacy,Pizza Place,French Restaurant
245,NYC,4,Staten Island,Arlington,Bus Stop,Caribbean Restaurant,Intersection,French Restaurant,Fish & Chips Shop
246,NYC,4,Staten Island,Arrochar,Bus Stop,Italian Restaurant,Deli / Bodega,Pizza Place,Hotel
248,NYC,4,Staten Island,Bloomfield,Park,Discount Store,Theme Park,Bus Stop,Recreation Center
250,NYC,4,Staten Island,Butler Manor,Baseball Field,Pool,Convenience Store,Bus Stop,Fish & Chips Shop
258,NYC,4,Staten Island,Elm Park,Ice Cream Shop,Bus Stop,Pizza Place,American Restaurant,Deli / Bodega
261,NYC,4,Staten Island,Fox Hills,Bus Stop,American Restaurant,BBQ Joint,Deli / Bodega,Bar


#### Cluster 5 consist of Neighborhoods with Bus Stops as the most common venues

## Let's bring to play our customer successful venues information

In [65]:
venues_comb_sorted.loc[venues_comb_sorted.Neighborhood.isin(['Bedford Stuyvesant','Flatbush', 'Financial District', 'Boerum Hill', 'North Side','Hamilton Heights', 'Manhattanville', 'Murray Hill', 'Hunters Point',\
                                                             'Long Island City', 'Arden Heights']), venues_comb_sorted.columns[[0]+[1] + [2] +[3]+ list(range(6, venues_comb_sorted.shape[1]))]]

,City,ClusterLabel,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
54,NYC,0,Brooklyn,Bedford Stuyvesant,Café,Coffee Shop,Pizza Place,Bar,Vietnamese Restaurant
57,NYC,0,Brooklyn,Boerum Hill,Coffee Shop,Dance Studio,French Restaurant,Spa,Sandwich Place
80,NYC,0,Brooklyn,Flatbush,Bakery,Coffee Shop,Caribbean Restaurant,Mexican Restaurant,Chinese Restaurant
101,NYC,0,Brooklyn,North Side,Coffee Shop,Pizza Place,Bar,Jewelry Store,American Restaurant
131,NYC,0,Manhattan,Financial District,Coffee Shop,Hotel,Pizza Place,Wine Shop,Gym
135,NYC,0,Manhattan,Hamilton Heights,Café,Pizza Place,Mexican Restaurant,Deli / Bodega,Coffee Shop
143,NYC,0,Manhattan,Manhattanville,Coffee Shop,Park,Mexican Restaurant,Seafood Restaurant,Fried Chicken Joint
148,NYC,0,Manhattan,Murray Hill,Coffee Shop,Japanese Restaurant,Hotel,Sandwich Place,French Restaurant
198,NYC,0,Queens,Hunters Point,Café,Italian Restaurant,Japanese Restaurant,Wine Shop,Thai Restaurant
209,NYC,0,Queens,Long Island City,Coffee Shop,Hotel,Bar,Pizza Place,Café


We are looking for Toronto Neighborhoods with similar characteristc, therefore in Cluster 1, we'll ignore result in Cluster 5 since looks like an anomaly. Look for 1st Common Venue Coffee Shop or Cafe, and with a the other 4 most popular also in food related business, such as Pizza Place, Bakery or Restaurant

In [66]:
venues_comb_sorted.loc[(venues_comb_sorted['ClusterLabel'] == 0) & (venues_comb_sorted['City']=='Toronto')\
                       & (venues_comb_sorted['1st Most Common Venue'].isin(['Coffee Shop', 'Café']))\
                       & (venues_comb_sorted['2nd Most Common Venue'].isin(['Café', 'Restaurant', 'Pizza Place', 'Bakery']))\
                        & (venues_comb_sorted['3rd Most Common Venue'].isin(['Café', 'Restaurant', 'Pizza Place', 'Bakery']))\
                       , venues_comb_sorted.columns[[0]+ [2] +[3]+ list(range(6, venues_comb_sorted.shape[1]))]]

,City,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
316,Toronto,Downtown Toronto,"Cabbagetown,St. James Town",Coffee Shop,Restaurant,Café,Italian Restaurant,Pizza Place
321,Toronto,Downtown Toronto,"Commerce Court,Victoria Hotel",Coffee Shop,Restaurant,Café,Hotel,American Restaurant
395,Toronto,West Toronto,"Brockton,Exhibition Place,Parkdale Village",Coffee Shop,Café,Restaurant,Sandwich Place,Bakery


## Results: The Neighborhoods in the above list fit the pattern of the already know succesful venues, these will be our recommended places to initiate a new French Coffee venues

### Let's put it in a map for more visual delivery

In [58]:
Toronto_rec = venues_comb_sorted.loc[(venues_comb_sorted['ClusterLabel'] == 0) & (venues_comb_sorted['City']=='Toronto')\
                       & (venues_comb_sorted['1st Most Common Venue'].isin(['Coffee Shop', 'Café'])\
                       & (venues_comb_sorted['2nd Most Common Venue'].isin(['Café', 'Restaurant', 'Pizza Place', 'Bakery']))\
                        & (venues_comb_sorted['3rd Most Common Venue'].isin(['Café', 'Restaurant', 'Pizza Place', 'Bakery']))].reset_index(drop=True)
Toronto_rec.head()

,City,ClusterLabel,Borough,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Toronto,0,Downtown Toronto,"Cabbagetown,St. James Town",43.668160,-79.366602,Coffee Shop,Restaurant,Café,Italian Restaurant,Pizza Place
1,Toronto,0,Downtown Toronto,"Commerce Court,Victoria Hotel",43.648205,-79.378793,Coffee Shop,Restaurant,Café,Hotel,American Restaurant
2,Toronto,0,West Toronto,"Brockton,Exhibition Place,Parkdale Village",43.639410,-79.424362,Coffee Shop,Café,Restaurant,Sandwich Place,Bakery


In [63]:

address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_ontario")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

map_toronto_rec = folium.Map(location=[latitude, longitude], zoom_start=12)

for lat, long, borough, neigh in zip(Toronto_rec['Neighborhood Latitude'], Toronto_rec['Neighborhood Longitude'], Toronto_rec['Borough'], Toronto_rec['Neighborhood']):
    label = '{}, {}'.format(neigh, borough)
    popup = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=popup,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_rec)
    
map_toronto_rec